In [1]:
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CONDA_PREFIX/lib/
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

# TensorFlow e tf.keras
import tensorflow as tf
from tensorflow import keras
import pandas as pd
# Bibliotecas Auxiliares
import seaborn as sns
import numpy as np
import collections
import matplotlib.pyplot as plt
import nest_asyncio
nest_asyncio.apply()
import tensorflow_federated as tff
np.random.seed(0)
from collections import Counter

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from matplotlib import pyplot
from tensorflow import keras

import collections
import functools
import os
import time
from scipy import stats

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter


print(tf.__version__)

/bin/bash: /home/guilherme/cpu-tensorflow-marcelo/nvidia-smi/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)


2023-03-03 14:15:42.027197: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-03 14:15:42.220420: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-03 14:15:42.756277: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/guilherme/cpu-tensorflow-marcelo/nvidia-smi/envs/tf/lib/
2023-03-03 14:15:42.758619: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7';

2.10.0


In [2]:
# y_test     = Array with real values
# yhat_probs = Array with predicted values
def printMetrics(y_test,yhat_probs):
    # predict crisp classes for test set deprecated
    #yhat_classes = model.predict_classes(X_test, verbose=0)
    #yhat_classes = np.argmax(yhat_probs,axis=1)
    yhat_classes = yhat_probs.round()
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(y_test, yhat_classes)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(y_test, yhat_classes)
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(y_test, yhat_classes)
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test, yhat_classes)
    print('F1 score: %f' % f1)
    # kappa
    kappa = cohen_kappa_score(y_test, yhat_classes)
    print('Cohens kappa: %f' % kappa)
    # ROC AUC
    auc = roc_auc_score(y_test, yhat_probs)
    print('ROC AUC: %f' % auc)
    # confusion matrix
    print("\Confusion Matrix")
    matrix = confusion_matrix(y_test, yhat_classes)
    print(matrix)
    
    results = dict()
    results['accuracy'] = accuracy
    results['precision'] = precision
    results['recall'] = recall
    results['f1_score'] = f1
    results['cohen_kappa_score'] = kappa
    results['roc_auc_score'] = auc
    return results

def showGlobalMetrics(metrics):
    accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score = 0,0,0,0,0,0
    for metric in metrics:
        accuracy = accuracy + metric['accuracy']
        precision = precision + metric['precision']
        recall = recall + metric['recall']
        f1_score = f1_score + metric['f1_score']
        cohen_kappa_score = cohen_kappa_score + metric['cohen_kappa_score']
        roc_auc_score = roc_auc_score + metric['roc_auc_score']
        
    # mean
    size = len(metrics)
    print(size)
    accuracy = accuracy / size
    precision = precision / size
    recall = recall / size
    f1_score = f1_score / size
    cohen_kappa_score = cohen_kappa_score / size
    roc_auc_score = roc_auc_score / size
    
    #show:\
    print("accuracy: ",accuracy)
    print("precision: ",precision)
    print("recall: ",recall)
    print("f1_score: ",f1_score)
    print("cohen_kappa_score: ",cohen_kappa_score)
    print("roc_auc_score: ",roc_auc_score)
    
def create_dataset_time_series_with_one_output(X, y, window_time_steps=1, shift_step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - window_time_steps, shift_step):
        v = X.iloc[i:(i + window_time_steps)].values
        labels = y.iloc[i: i + window_time_steps]
        Xs.append(v)        
        ys.append(stats.mode(labels)[0][0])
        
    if len(y.columns) == 1:
        return np.array(Xs), np.array(ys).reshape(-1, 1)
    else:
        return np.array(Xs), np.array(ys).reshape(-1, len(y.columns))

In [3]:
# configs
EPOCHS = 10
NUM_EPOCHS = EPOCHS
BATCH_SIZE = 32

TIME_SERIES_SIZE = 4   # Determines the window size. Ex (4,9)
TIME_STEP_SHIFT  = 1   # Determines specifies the number of steps to move the window forward at each iteration.

baseFolder = "../data_2019_processed/"

# selected features
inputFeatures = ["activity","location","day_of_week",
                 "light","phone_lock","proximity",
                 "sound","time_to_next_alarm", "minutes_day"]
# outputs
outputClasses = ["awake","asleep"]
#outputClasses = ["class"]

In [4]:
# test data comprising 25% of the data. It must be fixed to all models being evaluated
X_test  = pd.read_csv(baseFolder+"test/allData-classification-numeric-normalized.csv")

# input folder
inputFolders = baseFolder

# client datasets used on the training process
dsTrain =  ['0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs',
            '0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA', 
            '2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0', 
            '2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys', 
            #['5FLZBTVAPwdq9QezHE2sVCJIs7p+r6mCemA2gp9jATk'], #does not have the file
            '7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA', 
            'a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4', 
            'ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc', 
            'Ch3u5Oaz96VSrQbf0z31X6jEIbeIekkC0mwPzCdeJ1U', 
            'CH8f0yZkZL13zWuE9ks1CkVJRVrr+jsGdUXHrZ6YeA', 
            'DHO1K4jgiwZJOfQTrxvKE2vn7hkjamigroGD5IaeRc', 
            'DHPqzSqSttiba1L3BD1cptNJPjSxZ8rXxF9mY3za6WA', 
            'dQEFscjqnIlug8Tgq97JohhSQPG2DEOWJqS86wCrcY', 
            'HFvs2CohmhHte+AaCzFasjzegGzxZKPhkrX23iI6Xo', 
            'jgB9E8v3Z6PKdTRTCMAijBllA9YEMtrmHbe4qsbmJWw', 
            'JkY++R7E8myldLN3on6iQ78Ee78zCbrLuggfwGju3I', 
            'K4SLohf+TN1Ak8Dn8iE3Lme7rEMPISfppB2sXfHX8', 
            'oGaWetJJJEWHuvYdWYo826SQxfhCExVVQ2da8LE1Y7Q', 
            'pyt24oiDAHsmgWMvkFKz2fn2pwcHiXchd6KchLM', 
            #['PZCf1nfvhR+6fk+7+sPNMYOgb8BAMmtQtfoRS83Suc'], 
            'QUNCATForxzK0HHw46LrGOMWh0eVA8Y5XWEiUXX+cQ', 
            #['rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw'], 
            #['RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI'], 
            'SH3kQeyd5volraxw8vOyhlowNqWBPr1IJ9URNXUL4'] 
            #['VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is'], 
            #['Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw'], 
            #['XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA'], 
            #['YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw'], 
            #['ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM'], 
            #['ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY']]
            
dsTrain = ['0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs',
            '0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA', 
            '2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0', 
            '2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys', 
            #['5FLZBTVAPwdq9QezHE2sVCJIs7p+r6mCemA2gp9jATk'], #does not have the file
            '7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA', 
            'a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4', 
            'ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc', 
            'Ch3u5Oaz96VSrQbf0z31X6jEIbeIekkC0mwPzCdeJ1U', 
            'CH8f0yZkZL13zWuE9ks1CkVJRVrr+jsGdUXHrZ6YeA', 
            'DHO1K4jgiwZJOfQTrxvKE2vn7hkjamigroGD5IaeRc', 
            'DHPqzSqSttiba1L3BD1cptNJPjSxZ8rXxF9mY3za6WA', 
            'dQEFscjqnIlug8Tgq97JohhSQPG2DEOWJqS86wCrcY', 
            'HFvs2CohmhHte+AaCzFasjzegGzxZKPhkrX23iI6Xo', 
            'jgB9E8v3Z6PKdTRTCMAijBllA9YEMtrmHbe4qsbmJWw', 
            'JkY++R7E8myldLN3on6iQ78Ee78zCbrLuggfwGju3I', 
            'K4SLohf+TN1Ak8Dn8iE3Lme7rEMPISfppB2sXfHX8', 
            'oGaWetJJJEWHuvYdWYo826SQxfhCExVVQ2da8LE1Y7Q', 
            'pyt24oiDAHsmgWMvkFKz2fn2pwcHiXchd6KchLM', 
            'PZCf1nfvhR+6fk+7+sPNMYOgb8BAMmtQtfoRS83Suc', 
            'QUNCATForxzK0HHw46LrGOMWh0eVA8Y5XWEiUXX+cQ', 
            'rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw', 
            'RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI', 
            'SH3kQeyd5volraxw8vOyhlowNqWBPr1IJ9URNXUL4', 
            'VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is', 
            'Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw', 
            'XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA', 
            'YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw', 
            'ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM', 
            'ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY']




In [5]:
# load cliend data
clientList = []

# balancing classes
#oversample = RandomOverSampler(sampling_strategy='auto') #minority
#undersample = RandomUnderSampler(sampling_strategy='majority')

for i in range(0,len(dsTrain)):
    print (i," ", str(inputFolders)+"student_"+dsTrain[i]+"_numeric.csv") #_numeric
    # load client data
    dataset = pd.read_csv(inputFolders+"student_"+dsTrain[i]+"_numeric.csv")
    
    # print(dataset)
    y_train = dataset['class'].copy()
    
    # does not add datasets that dont have instances from both classes
    if y_train.sum() != 0 and (y_train.sum() != len(y_train)):
        X_train = dataset
        # balancing classes
        #X_train, y_train = oversample.fit_resample(dataset, y_train)
        #X_train, y_train = undersample.fit_resample(dataset, y_train)
        
        X_train['class'] = y_train
        
        clientList.append(X_train)
        
print("Total",(len(clientList)))

0   ../data_2019_processed/student_0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs_numeric.csv
1   ../data_2019_processed/student_0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA_numeric.csv
2   ../data_2019_processed/student_2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0_numeric.csv
3   ../data_2019_processed/student_2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys_numeric.csv
4   ../data_2019_processed/student_7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA_numeric.csv
5   ../data_2019_processed/student_a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4_numeric.csv
6   ../data_2019_processed/student_ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc_numeric.csv
7   ../data_2019_processed/student_Ch3u5Oaz96VSrQbf0z31X6jEIbeIekkC0mwPzCdeJ1U_numeric.csv
8   ../data_2019_processed/student_CH8f0yZkZL13zWuE9ks1CkVJRVrr+jsGdUXHrZ6YeA_numeric.csv
9   ../data_2019_processed/student_DHO1K4jgiwZJOfQTrxvKE2vn7hkjamigroGD5IaeRc_numeric.csv
10   ../data_2019_processed/student_DHPqzSqSttiba1L3BD1cptNJPjSxZ8rXxF9mY3za6WA_numeric.csv
11   ../

In [6]:
# undestand the dataset by looking on their infos
print(X_test.info())

X_test

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136286 entries, 0 to 136285
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            136286 non-null  float64
 1   location            136286 non-null  float64
 2   timestamp           136286 non-null  float64
 3   day_of_week         136286 non-null  float64
 4   light               136286 non-null  float64
 5   phone_lock          136286 non-null  float64
 6   proximity           136286 non-null  float64
 7   sound               136286 non-null  float64
 8   time_to_next_alarm  136286 non-null  float64
 9   minutes_day         136286 non-null  float64
 10  timestamp_text      136286 non-null  object 
 11  class               136286 non-null  object 
dtypes: float64(10), object(2)
memory usage: 12.5+ MB
None


,activity,location,timestamp,day_of_week,light,phone_lock,proximity,sound,time_to_next_alarm,minutes_day,timestamp_text,class
0,0.00,1.0,0.000647,1.000000,0.000000,0.0,1.0,0.655565,0.906151,0.115358,2018-05-15 02:46:57,asleep
1,0.25,1.0,0.000647,1.000000,0.000000,0.0,1.0,0.567296,0.906052,0.116053,2018-05-15 02:47:27,asleep
2,0.25,1.0,0.000647,1.000000,0.000000,0.0,1.0,0.567296,0.906052,0.116053,2018-05-15 02:47:57,asleep
3,0.25,1.0,0.000647,1.000000,0.000000,0.0,1.0,0.660604,0.905952,0.116748,2018-05-15 02:48:28,asleep
4,0.25,1.0,0.000647,1.000000,0.000000,0.0,1.0,0.660604,0.905952,0.116748,2018-05-15 02:48:57,asleep
...,...,...,...,...,...,...,...,...,...,...,...,...
136281,0.25,1.0,0.007015,0.166667,0.000056,1.0,1.0,0.000000,0.000099,0.510076,2018-06-13 12:14:37,awake
136282,0.25,1.0,0.007015,0.166667,0.000078,1.0,1.0,0.000000,0.000694,0.512856,2018-06-13 12:18:08,awake
136283,0.25,1.0,0.007015,0.166667,0.000078,1.0,1.0,0.000000,0.000595,0.513551,2018-06-13 12:19:08,awake
136284,0.25,1.0,0.007016,0.166667,0.000085,1.0,1.0,0.000000,0.000595,0.513551,2018-06-13 12:19:38,awake


In [7]:
# one-hot encoding function
def transform_output_nominal_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data['awake']
    dataset['asleep'] = one_hot_encoded_data['asleep']
    
    return dataset

# one-hot encoding function
def transform_output_numerical_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data[0]
    dataset['asleep'] = one_hot_encoded_data[1]
    
    return dataset

# transform output to one_hot_encoding for the testing dataset
X_test = transform_output_nominal_class_into_one_hot_encoding(X_test)

# transform output to one_hot_encoding for the input dataset
for i in range(0,len(clientList)):
    clientList[i] = transform_output_numerical_class_into_one_hot_encoding(clientList[i])
    #print (clientList[i])

In [8]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136286 entries, 0 to 136285
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            136286 non-null  float64
 1   location            136286 non-null  float64
 2   timestamp           136286 non-null  float64
 3   day_of_week         136286 non-null  float64
 4   light               136286 non-null  float64
 5   phone_lock          136286 non-null  float64
 6   proximity           136286 non-null  float64
 7   sound               136286 non-null  float64
 8   time_to_next_alarm  136286 non-null  float64
 9   minutes_day         136286 non-null  float64
 10  timestamp_text      136286 non-null  object 
 11  class               136286 non-null  object 
 12  awake               136286 non-null  uint8  
 13  asleep              136286 non-null  uint8  
dtypes: float64(10), object(2), uint8(2)
memory usage: 12.7+ MB


In [9]:
def transform_data_type(dataframe):
    
    # transform inputs
    for column in inputFeatures:
        dataframe[column] = dataframe[column].astype('float32')
    
    # transform outputs
    for column in outputClasses:
        dataframe[column] = dataframe[column].astype('float32')
    
    return dataframe

# transforms the data
X_test = transform_data_type(X_test)

X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136286 entries, 0 to 136285
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            136286 non-null  float32
 1   location            136286 non-null  float32
 2   timestamp           136286 non-null  float64
 3   day_of_week         136286 non-null  float32
 4   light               136286 non-null  float32
 5   phone_lock          136286 non-null  float32
 6   proximity           136286 non-null  float32
 7   sound               136286 non-null  float32
 8   time_to_next_alarm  136286 non-null  float32
 9   minutes_day         136286 non-null  float32
 10  timestamp_text      136286 non-null  object 
 11  class               136286 non-null  object 
 12  awake               136286 non-null  float32
 13  asleep              136286 non-null  float32
dtypes: float32(11), float64(1), object(2)
memory usage: 8.8+ MB


In [10]:
# selects the data to train and test
X_test_data = X_test[inputFeatures]
y_test_label = X_test[outputClasses]

# transform to time series
X_test_data, y_test_label = create_dataset_time_series_with_one_output(    #timestamp
    X_test_data, 
    y_test_label, 
    TIME_SERIES_SIZE, 
    TIME_STEP_SHIFT
)

# transtorm data to tensor slices
client_test_dataset = tf.data.Dataset.from_tensor_slices((X_test_data, y_test_label))
#client_test_dataset = tf.data.Dataset.from_tensor_slices((X_test_data.values, y_test_label.values))

#client_test_dataset = client_test_dataset.repeat(NUM_EPOCHS).batch(BATCH_SIZE, drop_remainder=True)
client_test_dataset = client_test_dataset.repeat(NUM_EPOCHS).batch(BATCH_SIZE)

print(client_test_dataset.element_spec)
client_test_dataset

2023-03-03 14:16:05.184785: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


(TensorSpec(shape=(None, 4, 9), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))


2023-03-03 14:16:05.213012: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-03 14:16:05.213451: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-03 14:16:05.213996: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-03 14:16:05.217269: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built witho

<BatchDataset element_spec=(TensorSpec(shape=(None, 4, 9), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))>

In [11]:
federated_training_data = []
# transform the data
for i in range(0,len(clientList)):
    # selects the data to train and test
    data   = clientList[i][inputFeatures]
    labels = clientList[i][outputClasses]
    # transform data to float32
    #data = transform_data_type(data)

    # transform to time series
    data, labels = create_dataset_time_series_with_one_output(    #timestamp
        data, 
        labels, 
        TIME_SERIES_SIZE, 
        TIME_STEP_SHIFT
    )

    # transform the data to tensor slices
    client_train_dataset = tf.data.Dataset.from_tensor_slices((data, labels))
    #client_train_dataset = tf.data.Dataset.from_tensor_slices((data.values, labels.values))
    # apply the configs
    client_train_dataset = client_train_dataset.repeat(NUM_EPOCHS).batch(BATCH_SIZE)
    # transform the data to
    federated_training_data.append(client_train_dataset)

In [12]:
def create_keras_model():
    model = keras.Sequential()
    model.add(LSTM(128,input_shape=(4, 9,))) #return_sequences=True, 
    #model.add(LSTM(128,input_shape=(2, 9,)))
    #model.add(keras.layers.Dropout(rate=0.5))
    #model.add(LSTM(64,activation="tanh"))
    model.add(keras.layers.Dropout(rate=0.5))
    model.add(keras.layers.Dense(2, activation='softmax'))#softmax,sigmoid
    
    return model

In [13]:
keras_model = create_keras_model()
#keras_model.summary()
keras_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               70656     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 2)                 258       
                                                                 
Total params: 70,914
Trainable params: 70,914
Non-trainable params: 0
_________________________________________________________________


In [14]:
def model_fn():
    # We _must_ create a new model here, and _not_ capture it from an external
    # scope. TFF will call this within different graph contexts.
    #keras_model = create_keras_model()
    keras_model = create_keras_model()
    return tff.learning.from_keras_model(
      keras_model,
      input_spec=client_train_dataset.element_spec,
      loss=tf.keras.losses.CategoricalCrossentropy(), #BinaryCrossentropy
      metrics=[tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.Precision()])

In [15]:
fed_avg_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),#client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

In [16]:
print(fed_avg_process.initialize.type_signature.formatted_representation())

( -> <
  global_model_weights=<
    trainable=<
      float32[9,512],
      float32[128,512],
      float32[512],
      float32[128,2],
      float32[2]
    >,
    non_trainable=<>
  >,
  distributor=<>,
  client_work=<>,
  aggregator=<
    value_sum_process=<>,
    weight_sum_process=<>
  >,
  finalizer=<
    int64
  >
>@SERVER)


In [17]:
state = fed_avg_process.initialize()

In [18]:
result = fed_avg_process.next(state, federated_training_data[0:10])
state = result.state
metrics = result.metrics
print('round  1, metrics={}'.format(metrics))

2023-03-03 14:17:38.278639: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


round  1, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.8344904), ('precision', 0.8331797), ('loss', 0.38585487), ('num_examples', 1909660), ('num_batches', 59681)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])


In [19]:
#def keras_evaluate(state, round_num):
    # Take our global model weights and push them back into a Keras model to
    # use its standard `.evaluate()` method.
keras_model = create_keras_model()


keras_model.compile(
  loss=tf.keras.losses.CategoricalCrossentropy(),
  metrics=[tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.Precision()])

# get neural network weights
weights = fed_avg_process.get_model_weights(state)
weights.assign_weights_to(keras_model)


#tttt.model.assign_weights_to(keras_model)
yhat_probs = keras_model.predict(X_test_data)

maxX = yhat_probs.max()
minX = yhat_probs.min()
avgX = yhat_probs.mean()

print(maxX,minX,avgX)
print(yhat_probs)
# predict crisp classes for test set deprecated
#printMetrics(y_test,yhat_probs)
test = list()

xx = yhat_probs.round()

y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

print('')
print('awake')
res = printMetrics(y_test_label_label['awake'],y_test2['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

2023-03-03 14:43:14.469808: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8201


4259/4259 [==============================] - 15s 2ms/step
0.9813663 0.018633725 0.5
[[0.8822726  0.11772738]
 [0.88438004 0.11562001]
 [0.8800734  0.11992665]
 ...
 [0.78117085 0.21882914]
 [0.7812719  0.21872818]
 [0.7813479  0.21865211]]

awake
Accuracy: 0.721034
Precision: 0.718250
Recall: 0.991003
F1 score: 0.832864
Cohens kappa: 0.104800
ROC AUC: 0.538981
\Confusion Matrix
[[ 3539 37158]
 [  860 94725]]

asleep
Accuracy: 0.722641
Precision: 0.803137
Recall: 0.087308
F1 score: 0.157495
Cohens kappa: 0.105405
ROC AUC: 0.539135
\Confusion Matrix
[[94950   866]
 [36933  3533]]

Global
2
accuracy:  0.7218378069003977
precision:  0.7606936719453421
recall:  0.5391553178965083
f1_score:  0.495179554623135
cohen_kappa_score:  0.10510221583936213
roc_auc_score:  0.5390580515219106


In [20]:
y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

print('')
print('awake')
res = printMetrics(y_test_label_label['awake'],y_test2['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)


awake
Accuracy: 0.721034
Precision: 0.718250
Recall: 0.991003
F1 score: 0.832864
Cohens kappa: 0.104800
ROC AUC: 0.538981
\Confusion Matrix
[[ 3539 37158]
 [  860 94725]]

asleep
Accuracy: 0.722641
Precision: 0.803137
Recall: 0.087308
F1 score: 0.157495
Cohens kappa: 0.105405
ROC AUC: 0.539135
\Confusion Matrix
[[94950   866]
 [36933  3533]]

Global
4
accuracy:  0.7218378069003978
precision:  0.7606936719453421
recall:  0.5391553178965083
f1_score:  0.495179554623135
cohen_kappa_score:  0.10510221583936213
roc_auc_score:  0.5390580515219106


In [21]:
result = fed_avg_process.next(state, federated_training_data[10:19])
state = result.state
metrics = result.metrics
print('round  2, metrics={}'.format(metrics))

round  2, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.85207903), ('precision', 0.8508464), ('loss', 0.34749988), ('num_examples', 2050980), ('num_batches', 64098)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])


In [22]:
#def keras_evaluate(state, round_num):
    # Take our global model weights and push them back into a Keras model to
    # use its standard `.evaluate()` method.
keras_model = create_keras_model()


keras_model.compile(
  loss=tf.keras.losses.CategoricalCrossentropy(),
  metrics=[tf.keras.metrics.CategoricalAccuracy()])

# get neural network weights
weights = fed_avg_process.get_model_weights(state)
weights.assign_weights_to(keras_model)


#tttt.model.assign_weights_to(keras_model)
yhat_probs = keras_model.predict(X_test_data)

maxX = yhat_probs.max()
minX = yhat_probs.min()
avgX = yhat_probs.mean()

print(maxX,minX,avgX)
print(yhat_probs)
# predict crisp classes for test set deprecated
#printMetrics(y_test,yhat_probs)
test = list()

xx = yhat_probs.round()

y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

print('')
print('awake')
res = printMetrics(y_test_label_label['awake'],y_test2['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

4259/4259 [==============================] - 10s 2ms/step
0.99573225 0.004267701 0.5
[[0.90544015 0.09455983]
 [0.9109226  0.08907737]
 [0.9031716  0.09682841]
 ...
 [0.88010997 0.11988997]
 [0.880334   0.11966599]
 [0.88050455 0.11949547]]

awake
Accuracy: 0.702037
Precision: 0.702578
Recall: 0.997405
F1 score: 0.824426
Cohens kappa: 0.007965
ROC AUC: 0.502855
\Confusion Matrix
[[  338 40359]
 [  248 95337]]

asleep
Accuracy: 0.703717
Precision: 0.575085
Recall: 0.008328
F1 score: 0.016418
Cohens kappa: 0.008009
ROC AUC: 0.502865
\Confusion Matrix
[[95567   249]
 [40129   337]]

Global
2
accuracy:  0.7028771224372992
precision:  0.6388315726218775
recall:  0.5028667148450788
f1_score:  0.42042194995735616
cohen_kappa_score:  0.007986797601246376
roc_auc_score:  0.5028599948191924


In [23]:
for i in range(0,2):
    result = fed_avg_process.next(state, federated_training_data[0:19])
    state = result.state
    metrics = result.metrics
    print('round  {}-{}, metrics={}'.format(i*2+2,i*2+4,metrics))
    
    print('')
    #def keras_evaluate(state, round_num):
        # Take our global model weights and push them back into a Keras model to
        # use its standard `.evaluate()` method.
    keras_model = create_keras_model()


    keras_model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.CategoricalAccuracy()])

    # get neural network weights
    weights = fed_avg_process.get_model_weights(state)
    weights.assign_weights_to(keras_model)


    #tttt.model.assign_weights_to(keras_model)
    yhat_probs = keras_model.predict(X_test_data)

    maxX = yhat_probs.max()
    minX = yhat_probs.min()
    avgX = yhat_probs.mean()

    print(maxX,minX,avgX)
    print(yhat_probs)
    # predict crisp classes for test set deprecated
    #printMetrics(y_test,yhat_probs)
    test = list()

    xx = yhat_probs.round()

    y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
    y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

    print('')
    print('awake')
    res = printMetrics(y_test_label_label['awake'],y_test2['awake'])
    test.append(res)
    print('')
    print('asleep')
    res = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
    test.append(res)
    print('')
    print('Global')
    showGlobalMetrics(test)
    print('')
    print('')
    print('')

round  2-4, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.8415491), ('precision', 0.8402831), ('loss', 0.36699146), ('num_examples', 3960640), ('num_batches', 123779)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])

4259/4259 [==============================] - 9s 2ms/step
0.9913181 0.008681943 0.5
[[0.8763326  0.12366738]
 [0.88253236 0.11746759]
 [0.87401515 0.12598479]
 ...
 [0.83723706 0.16276288]
 [0.8374307  0.16256928]
 [0.8375677  0.16243236]]

awake
Accuracy: 0.712721
Precision: 0.714242
Recall: 0.984150
F1 score: 0.827749
Cohens kappa: 0.079666
ROC AUC: 0.529682
\Confusion Matrix
[[ 3061 37636]
 [ 1515 94070]]

asleep
Accuracy: 0.714269
Precision: 0.666740
Recall: 0.075397
F1 score: 0.135474
Cohens kappa: 0.079966
ROC AUC: 0.529740
\Confusion Matrix
[[94291  1525]
 [37415  3051]]

Global
2
accuracy:  0.71349481222758

In [24]:
for i in range(2,4):
    result = fed_avg_process.next(state, federated_training_data[0:19])
    state = result.state
    metrics = result.metrics
    print('round  {}-{}, metrics={}'.format(i*2+2,i*2+4,metrics))

    print('')
    #def keras_evaluate(state, round_num):
        # Take our global model weights and push them back into a Keras model to
        # use its standard `.evaluate()` method.
    keras_model = create_keras_model()


    keras_model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.CategoricalAccuracy()])

    # get neural network weights
    weights = fed_avg_process.get_model_weights(state)
    weights.assign_weights_to(keras_model)


    #tttt.model.assign_weights_to(keras_model)
    yhat_probs = keras_model.predict(X_test_data)

    maxX = yhat_probs.max()
    minX = yhat_probs.min()
    avgX = yhat_probs.mean()

    print(maxX,minX,avgX)
    print(yhat_probs)
    # predict crisp classes for test set deprecated
    #printMetrics(y_test,yhat_probs)
    test = list()

    xx = yhat_probs.round()

    y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
    y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

    print('')
    print('awake')
    res = printMetrics(y_test_label_label['awake'],y_test2['awake'])
    test.append(res)
    print('')
    print('asleep')
    res = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
    test.append(res)
    print('')
    print('Global')
    showGlobalMetrics(test)
    print('')
    print('')
    print('')

round  6-8, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.8425648), ('precision', 0.84129053), ('loss', 0.36608386), ('num_examples', 3960640), ('num_batches', 123779)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])

4259/4259 [==============================] - 15s 3ms/step
0.99010134 0.009898652 0.5
[[0.8692552  0.13074479]
 [0.87661386 0.12338614]
 [0.86785835 0.13214165]
 ...
 [0.8417765  0.15822357]
 [0.84194773 0.15805225]
 [0.84208447 0.1579155 ]]

awake
Accuracy: 0.715494
Precision: 0.716456
Recall: 0.983648
F1 score: 0.829056
Cohens kappa: 0.092624
ROC AUC: 0.534665
\Confusion Matrix
[[ 3487 37210]
 [ 1563 94022]]

asleep
Accuracy: 0.717028
Precision: 0.688317
Recall: 0.085899
F1 score: 0.152737
Cohens kappa: 0.092975
ROC AUC: 0.534736
\Confusion Matrix
[[94242  1574]
 [36990  3476]]

Global
2
accuracy:  0.71626113499

In [25]:
import datetime;
  
current_time = datetime.datetime.now()
  
time_stamp = current_time.timestamp()
print("Start timestamp:-", time_stamp,current_time)

Start timestamp:- 1677869886.7468 2023-03-03 18:58:06.746800


In [26]:
for i in range(4,6):
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i*2+2,"Start timestamp:", time_stamp,current_time)
    
    result = fed_avg_process.next(state, federated_training_data[0:19])
    state = result.state
    metrics = result.metrics
    print('round  {}-{}, metrics={}'.format(i*2+2,i*2+4,metrics))

    print('')
    #def keras_evaluate(state, round_num):
        # Take our global model weights and push them back into a Keras model to
        # use its standard `.evaluate()` method.
    keras_model = create_keras_model()


    keras_model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.CategoricalAccuracy()])

    # get neural network weights
    weights = fed_avg_process.get_model_weights(state)
    weights.assign_weights_to(keras_model)


    #tttt.model.assign_weights_to(keras_model)
    yhat_probs = keras_model.predict(X_test_data)

    maxX = yhat_probs.max()
    minX = yhat_probs.min()
    avgX = yhat_probs.mean()

    print(maxX,minX,avgX)
    print(yhat_probs)
    # predict crisp classes for test set deprecated
    #printMetrics(y_test,yhat_probs)
    test = list()

    xx = yhat_probs.round()

    y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
    y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

    print('')
    print('awake')
    res = printMetrics(y_test_label_label['awake'],y_test2['awake'])
    test.append(res)
    print('')
    print('asleep')
    res = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
    test.append(res)
    print('')
    print('Global')
    showGlobalMetrics(test)
    print('')
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i*2+2,"End timestamp:", time_stamp,current_time)
    print('')
    print('')

10 Start timestamp: 1677869949.412024 2023-03-03 18:59:09.412024
round  10-12, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.8452523), ('precision', 0.8439901), ('loss', 0.36216947), ('num_examples', 3960640), ('num_batches', 123779)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])

4259/4259 [==============================] - 10s 2ms/step
0.98856807 0.011431987 0.5
[[0.85414046 0.14585957]
 [0.86318874 0.13681124]
 [0.8535256  0.14647445]
 ...
 [0.83195233 0.1680476 ]
 [0.83212245 0.1678776 ]
 [0.8322661  0.16773391]]

awake
Accuracy: 0.724733
Precision: 0.729260
Recall: 0.966260
F1 score: 0.831196
Cohens kappa: 0.158440
ROC AUC: 0.561858
\Confusion Matrix
[[ 6408 34289]
 [ 3225 92360]]

asleep
Accuracy: 0.726149
Precision: 0.663241
Recall: 0.157886
F1 score: 0.255055
Cohens kappa: 0.159027
ROC AUC: 0.562015
\Confusion Matrix

In [27]:
for i in range(6,8):
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i*2+2,"Start timestamp:", time_stamp,current_time)
    
    result = fed_avg_process.next(state, federated_training_data[0:19])
    state = result.state
    metrics = result.metrics
    print('round  {}-{}, metrics={}'.format(i*2+2,i*2+4,metrics))

    print('')
    #def keras_evaluate(state, round_num):
        # Take our global model weights and push them back into a Keras model to
        # use its standard `.evaluate()` method.
    keras_model = create_keras_model()


    keras_model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.CategoricalAccuracy()])

    # get neural network weights
    weights = fed_avg_process.get_model_weights(state)
    weights.assign_weights_to(keras_model)


    #tttt.model.assign_weights_to(keras_model)
    yhat_probs = keras_model.predict(X_test_data)

    maxX = yhat_probs.max()
    minX = yhat_probs.min()
    avgX = yhat_probs.mean()

    print(maxX,minX,avgX)
    print(yhat_probs)
    # predict crisp classes for test set deprecated
    #printMetrics(y_test,yhat_probs)
    test = list()

    xx = yhat_probs.round()

    y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
    y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

    print('')
    print('awake')
    res = printMetrics(y_test_label_label['awake'],y_test2['awake'])
    test.append(res)
    print('')
    print('asleep')
    res = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
    test.append(res)
    print('')
    print('Global')
    showGlobalMetrics(test)
    print('')
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i*2+2,"End timestamp:", time_stamp,current_time)
    print('')
    print('')

14 Start timestamp: 1677876295.287352 2023-03-03 20:44:55.287352
round  14-16, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.84840226), ('precision', 0.8471487), ('loss', 0.3568586), ('num_examples', 3960640), ('num_batches', 123779)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])

4259/4259 [==============================] - 9s 2ms/step
0.9879844 0.012015647 0.5
[[0.8380231  0.16197689]
 [0.8495376  0.15046233]
 [0.83833677 0.16166326]
 ...
 [0.8180887  0.18191127]
 [0.81829    0.18171003]
 [0.81843585 0.18156414]]

awake
Accuracy: 0.730918
Precision: 0.741772
Recall: 0.945504
F1 score: 0.831338
Cohens kappa: 0.211620
ROC AUC: 0.586212
\Confusion Matrix
[[ 9235 31462]
 [ 5209 90376]]

asleep
Accuracy: 0.732202
Precision: 0.637427
Recall: 0.227524
F1 score: 0.335349
Cohens kappa: 0.212300
ROC AUC: 0.586434
\Confusion Matrix
[

KeyboardInterrupt: 

In [28]:
for i in range(7,8):
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i*2+2,"Start timestamp:", time_stamp,current_time)
    
    result = fed_avg_process.next(state, federated_training_data[0:19])
    state = result.state
    metrics = result.metrics
    print('round  {}-{}, metrics={}'.format(i*2+2,i*2+4,metrics))

    print('')
    #def keras_evaluate(state, round_num):
        # Take our global model weights and push them back into a Keras model to
        # use its standard `.evaluate()` method.
    keras_model = create_keras_model()


    keras_model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.CategoricalAccuracy()])

    # get neural network weights
    weights = fed_avg_process.get_model_weights(state)
    weights.assign_weights_to(keras_model)


    #tttt.model.assign_weights_to(keras_model)
    yhat_probs = keras_model.predict(X_test_data)

    maxX = yhat_probs.max()
    minX = yhat_probs.min()
    avgX = yhat_probs.mean()

    print(maxX,minX,avgX)
    print(yhat_probs)
    # predict crisp classes for test set deprecated
    #printMetrics(y_test,yhat_probs)
    test = list()

    xx = yhat_probs.round()

    y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
    y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

    print('')
    print('awake')
    res = printMetrics(y_test_label_label['awake'],y_test2['awake'])
    test.append(res)
    print('')
    print('asleep')
    res = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
    test.append(res)
    print('')
    print('Global')
    showGlobalMetrics(test)
    print('')
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i*2+2,"End timestamp:", time_stamp,current_time)
    print('')
    print('')

16 Start timestamp: 1677888838.546807 2023-03-04 00:13:58.546807
round  16-18, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.85085416), ('precision', 0.8496074), ('loss', 0.35291463), ('num_examples', 3960640), ('num_batches', 123779)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])

4259/4259 [==============================] - 11s 2ms/step
0.98855364 0.011446401 0.5
[[0.828876   0.17112406]
 [0.84279037 0.15720965]
 [0.8299557  0.17004427]
 ...
 [0.8119127  0.1880873 ]
 [0.81212    0.18788   ]
 [0.81228584 0.18771414]]

awake
Accuracy: 0.731021
Precision: 0.745117
Recall: 0.937030
F1 score: 0.830126
Cohens kappa: 0.222915
ROC AUC: 0.592099
\Confusion Matrix
[[10059 30638]
 [ 6019 89566]]

asleep
Accuracy: 0.732217
Precision: 0.623523
Recall: 0.247739
F1 score: 0.354591
Cohens kappa: 0.223465
ROC AUC: 0.592283
\Confusion Matri

In [29]:
for i in range(8,16):
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i*2+2,"Start timestamp:", time_stamp,current_time)
    
    result = fed_avg_process.next(state, federated_training_data[0:19])
    state = result.state
    metrics = result.metrics
    print('round  {}-{}, metrics={}'.format(i*2+2,i*2+4,metrics))

    print('')
    #def keras_evaluate(state, round_num):
        # Take our global model weights and push them back into a Keras model to
        # use its standard `.evaluate()` method.
    keras_model = create_keras_model()


    keras_model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.CategoricalAccuracy()])

    # get neural network weights
    weights = fed_avg_process.get_model_weights(state)
    weights.assign_weights_to(keras_model)


    #tttt.model.assign_weights_to(keras_model)
    yhat_probs = keras_model.predict(X_test_data)

    maxX = yhat_probs.max()
    minX = yhat_probs.min()
    avgX = yhat_probs.mean()

    print(maxX,minX,avgX)
    print(yhat_probs)
    # predict crisp classes for test set deprecated
    #printMetrics(y_test,yhat_probs)
    test = list()

    xx = yhat_probs.round()

    y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
    y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

    print('')
    print('awake')
    res = printMetrics(y_test_label_label['awake'],y_test2['awake'])
    test.append(res)
    print('')
    print('asleep')
    res = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
    test.append(res)
    print('')
    print('Global')
    showGlobalMetrics(test)
    print('')
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i*2+2,"End timestamp:", time_stamp,current_time)
    print('')
    print('')

18 Start timestamp: 1677892069.371432 2023-03-04 01:07:49.371432
round  18-20, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.85353935), ('precision', 0.85230845), ('loss', 0.34867692), ('num_examples', 3960640), ('num_batches', 123779)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])

4259/4259 [==============================] - 11s 2ms/step
0.9895176 0.010482413 0.5
[[0.82110554 0.17889448]
 [0.837641   0.16235895]
 [0.8231458  0.17685422]
 ...
 [0.8127625  0.18723753]
 [0.8129631  0.18703686]
 [0.81315446 0.18684554]]

awake
Accuracy: 0.732415
Precision: 0.748053
Recall: 0.932584
F1 score: 0.830188
Cohens kappa: 0.233747
ROC AUC: 0.597432
\Confusion Matrix
[[10674 30023]
 [ 6444 89141]]

asleep
Accuracy: 0.733538
Precision: 0.621276
Recall: 0.262813
F1 score: 0.369373
Cohens kappa: 0.234178
ROC AUC: 0.597576
\Confusion Matri

4259/4259 [==============================] - 11s 2ms/step
0.9962304 0.003769584 0.5
[[0.849222   0.15077807]
 [0.8694857  0.1305144 ]
 [0.85284084 0.1471592 ]
 ...
 [0.7562985  0.2437015 ]
 [0.7567326  0.24326734]
 [0.75717366 0.24282633]]

awake
Accuracy: 0.726816
Precision: 0.737526
Recall: 0.947816
F1 score: 0.829552
Cohens kappa: 0.192604
ROC AUC: 0.577785
\Confusion Matrix
[[ 8455 32242]
 [ 4988 90597]]

asleep
Accuracy: 0.728042
Precision: 0.626571
Recall: 0.208150
F1 score: 0.312490
Cohens kappa: 0.192981
ROC AUC: 0.577879
\Confusion Matrix
[[90796  5020]
 [32043  8423]]

Global
2
accuracy:  0.7274291542536799
precision:  0.6820489027307135
recall:  0.5779830659121388
f1_score:  0.5710206316826679
cohen_kappa_score:  0.19279234191471817
roc_auc_score:  0.5778322295903653

30 End timestamp: 1677914067.873753 2023-03-04 07:14:27.873753


32 Start timestamp: 1677914067.87381 2023-03-04 07:14:27.873810
round  32-34, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDi